# Notebook présentant le process ETL pour ce projet

Contenue du notebook :
- Extraction des données depuis la source
- Transformation des données
- Et chargement des données dans PowerBI

Source des données :
- **REI_2021.xlsx** : Ce fichier est le dataset de toutes les données correspondant à la taxe foncière des propriétés bâties.
- **TRACE_REI_2021.xlsx** : Ce fichier contient les informations relatives à toutes les colonnes du dataset.

### Installation des dépendances

In [ ]:
pip install openpyxl pandas duckdb  

In [5]:
import duckdb
import pandas as pd

### Identification des colonnes que l'on doit garder

In [6]:
import pandas as pd

df = pd.read_excel("TRACE_REI_2021.xlsx", usecols=['TITRE', 'OBSERVATIONS'])
df.columns.tolist()
df.head()

,TITRE,OBSERVATIONS
0,DEPARTEMENT,Code d'identification du département sur deux ...
1,DIRECTION,Donnée utilisée en interne par la Direction gé...
2,COMMUNE,Code officiel géographique d'identification d'...
3,COMMUNE RECENSEE (R si recensée),Indicateur de recensement. Prend la valeur R p...
4,Numéro national du groupement,Donnée d'identification de l'EPCI à fiscalité ...


In [7]:
cols = ['DEPARTEMENT',
 'COMMUNE',
 'Libellé commune',
 'FB - COMMUNE / MONTANT REEL',
 'FB - COMMUNE / BASE NETTE',
 'FB - COMMUNE / TAUX NET',
 'FB - COMMUNE / NOMBRE D\'ARTICLES']

# ETL

## 1. Extraction des données

On extrait les données accessibles depuis le fichier 'REI_2021.xlsx' en utilisant duckdb pour un chargement plus efficace et rapide.

In [8]:
query = f"""
    SELECT {', '.join([f'"{col}"' for col in cols])}
    FROM REI_2021.xlsx
"""

duckdf = duckdb.sql(query)
duckdf

┌─────────────┬─────────┬───────────────────────────┬─────────────────────────────┬───────────────────────────┬─────────────────────────┬──────────────────────────────────┐
│ DEPARTEMENT │ COMMUNE │      Libellé commune      │ FB - COMMUNE / MONTANT REEL │ FB - COMMUNE / BASE NETTE │ FB - COMMUNE / TAUX NET │ FB - COMMUNE / NOMBRE D'ARTICLES │
│   varchar   │ varchar │          varchar          │           double            │          double           │         double          │              double              │
├─────────────┼─────────┼───────────────────────────┼─────────────────────────────┼───────────────────────────┼─────────────────────────┼──────────────────────────────────┤
│ 01          │ 001     │ ABERGEMENT CLEMENCIAT     │                    135026.0 │                  546255.0 │      24.769999980926514 │                            319.0 │
│ 01          │ 002     │ ABERGEMENT DE VAREY       │                     57512.0 │                  212966.0 │                    27.0

On convertit le dataset duckdb en DataFrame (pandas) puis on affiche quelques informations supplémentaires sur l'ensemble de données.

In [9]:
df = duckdf.df()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34969 entries, 0 to 34968
Data columns (total 7 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   DEPARTEMENT                       34969 non-null  object 
 1   COMMUNE                           34969 non-null  object 
 2   Libellé commune                   34969 non-null  object 
 3   FB - COMMUNE / MONTANT REEL       34893 non-null  float64
 4   FB - COMMUNE / BASE NETTE         34893 non-null  float64
 5   FB - COMMUNE / TAUX NET           34969 non-null  float64
 6   FB - COMMUNE / NOMBRE D'ARTICLES  34893 non-null  float64
dtypes: float64(4), object(3)
memory usage: 1.9+ MB


## 2. Transformation des données

On commence par supprimer les données qui contiennent des valeurs nulles et/ou une base nette non positive.

In [10]:
df.dropna()
df = df[df['FB - COMMUNE / BASE NETTE'] > 0]

Durant l'exploration des données, nous avons pu voir que la colonne COMMUNE contient uniquement les 3 derniers chiffres du code INSEE (001 pour une ville dont le code est 57001). On va transformer cette colonne pour obtenir le code INSEE, ce qui permettra d'avoir un identifiant unique depuis cette colonne.

In [11]:
df['COMMUNE'] = df['DEPARTEMENT'].astype(str) + df['COMMUNE'].astype(str)

Maintenant que l'on a une colonne qui fait office d'identifiant unique pour chaque ville, on vérifie maintenant si il existe des doublons (chaque commune doit avoir une unique ligne de données).

In [12]:
df[df.duplicated('COMMUNE')]

,DEPARTEMENT,COMMUNE,Libellé commune,FB - COMMUNE / MONTANT REEL,FB - COMMUNE / BASE NETTE,FB - COMMUNE / TAUX NET,FB - COMMUNE / NOMBRE D'ARTICLES


Aucun doublon détecté.

Ensuite, on créer de nouvelles colonnes calculés. Ces nouvelles colonnes sont interessantes pour la mise en place de dashboard BI, comme le produit attendu pour chaque commune, ou encore l'écart relatif qu'il y a entre le produit réel et le produit attendu.

In [13]:
df['PRODUIT_ATTENDU'] = df['FB - COMMUNE / BASE NETTE'] * df['FB - COMMUNE / TAUX NET'] / 100

In [14]:
df['ECART_RELATIF'] = abs(df['FB - COMMUNE / MONTANT REEL'] - df['PRODUIT_ATTENDU']) / df['PRODUIT_ATTENDU']

In [15]:
df['ECART_RELATIF_NON_ABSOLU'] = (df['FB - COMMUNE / MONTANT REEL'] - df['PRODUIT_ATTENDU']) / df['PRODUIT_ATTENDU']

Pour terminer, nous allons renommer les colonnes pour une compréhension plus simple.

In [16]:
df.rename(columns={
    'COMMUNE' : 'CODE_INSEE',
    'Libellé commune' : 'NOM_COMMUNE',
    'FB - COMMUNE / MONTANT REEL': 'MONTANT_REEL',
    'FB - COMMUNE / BASE NETTE': 'BASE_NETTE',
    'FB - COMMUNE / TAUX NET': 'TAUX_NET',
    'FB - COMMUNE / NOMBRE D\'ARTICLES': 'NOMBRE_ARTICLES'
}, inplace=True)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34885 entries, 0 to 34968
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   DEPARTEMENT               34885 non-null  object 
 1   CODE_INSEE                34885 non-null  object 
 2   NOM_COMMUNE               34885 non-null  object 
 3   MONTANT_REEL              34885 non-null  float64
 4   BASE_NETTE                34885 non-null  float64
 5   TAUX_NET                  34885 non-null  float64
 6   NOMBRE_ARTICLES           34885 non-null  float64
 7   PRODUIT_ATTENDU           34885 non-null  float64
 8   ECART_RELATIF             34884 non-null  float64
 9   ECART_RELATIF_NON_ABSOLU  34884 non-null  float64
dtypes: float64(7), object(3)
memory usage: 2.9+ MB


## 3. Chargement des données dans PowerBI

Dans ce projet, nous accédons aux données depuis PowerBI en accédant au fichier `etl_output.csv`. 

Ainsi, pour cette étape, il suffit alors d'exporter le dataset au format csv sous le nom `etl_output.csv`.

In [18]:
df.to_csv('etl_output.csv', index=False)